## YOLO model training for License Plate Detection

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Task/SampleSubmission.csv')
print(df.head())

recog_anot = pd.read_csv('/content/drive/My Drive/Task/Licplatesrecognition_train.csv')
print(recog_anot.head())

detec_anot = pd.read_csv('/content/drive/My Drive/Task/Licplatesdetection_train.csv')
print(detec_anot.head())

          id    0    1    2    3    4    5    6    7    8    9
0  img_901_1  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1  img_901_2  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
2  img_901_3  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
3  img_901_4  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
4  img_901_5  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
    img_id      text
0    0.jpg  117T3989
1    1.jpg  128T8086
2   10.jpg   94T3458
3  100.jpg  133T6719
4  101.jpg   68T5979
    img_id  ymin  xmin  ymax  xmax
0    1.jpg   276    94   326   169
1   10.jpg   311   395   344   444
2  100.jpg   406   263   450   434
3  101.jpg   283   363   315   494
4  102.jpg   139    42   280   222


File extraction from google drive and unzipping the files


In [ ]:
import zipfile
import os

zip_files = [
    '/content/drive/My Drive/Task/test.zip',
    '/content/drive/My Drive/Task/Licplatesdetection_train.zip',
    '/content/drive/My Drive/Task/Licplatesrecognition_train.zip'
]

# Extract each zip file
for zip_path in zip_files:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        extract_path = os.path.join('/content', os.path.basename(zip_path).replace('.zip', ''))
        zip_ref.extractall(extract_path)
        print(f'Extracted {zip_path} to {extract_path}')


Extracted /content/drive/My Drive/Task/test.zip to /content/test
Extracted /content/drive/My Drive/Task/Licplatesdetection_train.zip to /content/Licplatesdetection_train
Extracted /content/drive/My Drive/Task/Licplatesrecognition_train.zip to /content/Licplatesrecognition_train


Data split for training and validation

In [ ]:
def split_data(source_dir, train_dir, val_dir, split_ratio=0.8):
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)

    images = [f for f in os.listdir(source_dir) if f.endswith(('.jpg'))]
    random.shuffle(images)

    split_index = int(len(images) * split_ratio)
    train_images, val_images = images[:split_index], images[split_index:]

    for img in train_images:
        shutil.move(os.path.join(source_dir, img), os.path.join(train_dir, img))
    for img in val_images:
        shutil.move(os.path.join(source_dir, img), os.path.join(val_dir, img))

    print(f"Split {len(train_images)} images into {train_dir} and {len(val_images)} into {val_dir}")


In [ ]:
import random
import shutil
split_data("/content/Licplatesdetection_train/license_plates_detection_train", "/content/Licplatesdetection_train/train/images", "/content/Licplatesdetection_train/val/images")



Split 720 images into /content/Licplatesdetection_train/train/images and 180 into /content/Licplatesdetection_train/val/images


In [ ]:
# Load annotation CSVs
detec_anot = pd.read_csv('/content/drive/My Drive/Task/Licplatesdetection_train.csv')
recog_anot = pd.read_csv('/content/drive/My Drive/Task/Licplatesrecognition_train.csv')

# Get train & val images
train_det_images = set(os.listdir("/content/Licplatesdetection_train/train/images"))
val_det_images = set(os.listdir("/content/Licplatesdetection_train/val/images"))


# Split detection labels
train_det_labels = detec_anot[detec_anot['img_id'].isin(train_det_images)]
val_det_labels = detec_anot[detec_anot['img_id'].isin(val_det_images)]

# Save new CSVs
train_det_labels.to_csv("/content/Licplatesdetection_train/train_labels.csv", index=False)
val_det_labels.to_csv("/content/Licplatesdetection_train/val_labels.csv", index=False)


print("Annotations split into train and val")


Annotations split into train and val


Creating YAML files for tolo model training

In [ ]:
import os
import pandas as pd
from PIL import Image

# Function to convert DETECTION annotations (Bounding Box Format) to YOLO format
def convert_detection_to_yolo(csv_path, output_dir, image_dir):
    os.makedirs(output_dir, exist_ok=True)
    df = pd.read_csv(csv_path)
    df.columns = df.columns.str.strip()  # Remove extra spaces from column names

    required_columns = {"img_id", "xmin", "ymin", "xmax", "ymax"}
    if not required_columns.issubset(df.columns):
        raise ValueError(f"Missing columns in {csv_path}. Expected {required_columns}")

    for _, row in df.iterrows():
        img_name = row["img_id"]
        img_path = os.path.join(image_dir, img_name)

        with Image.open(img_path) as img:
            img_width, img_height = img.size

        # Get bounding box values
        x_min, y_min, x_max, y_max = row["xmin"], row["ymin"], row["xmax"], row["ymax"]

        # Convert to YOLO format (normalize)
        width, height = x_max - x_min, y_max - y_min
        x_center = (x_min + width / 2) / img_width
        y_center = (y_min + height / 2) / img_height
        width /= img_width
        height /= img_height


        # Create YOLO label file
        label_path = os.path.join(output_dir, img_name.replace(".jpg", ".txt"))
        with open(label_path, "w") as f:
            f.write(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")



# Convert Detection Labels
convert_detection_to_yolo(
    "/content/Licplatesdetection_train/train_labels.csv",
    "/content/Licplatesdetection_train/train/labels",
    "/content/Licplatesdetection_train/train/images"
)

convert_detection_to_yolo(
    "/content/Licplatesdetection_train/val_labels.csv",
    "/content/Licplatesdetection_train/val/labels",
    "/content/Licplatesdetection_train/val/images"
)


print("Detection annotations successfully converted to YOLO format!")


Detection annotations successfully converted to YOLO format!


In [ ]:
# YAML for License Plate Detection
detection_yaml = """path: /content/Licplatesdetection_train  # Base dataset path

train: train/images  # Training images directory
val: val/images      # Validation images directory

# Class labels for detection
names:
  0: 'License_Plate'
"""

with open("license_plate_detection.yaml", "w") as f:
    f.write(detection_yaml)


print("YAML files created successfully!")


YAML files created successfully!


In [ ]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.2/949.2 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
from ultralytics import YOLO
import cv2
import os
import numpy as np


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLO Model training

In [ ]:
from ultralytics import YOLO

detection_model = YOLO("yolov8n.pt")

# Train the detection model
detection_model.train(data="license_plate_detection.yaml", epochs=16, imgsz=640, batch=16, workers=2)


100%|██████████| 6.25M/6.25M [00:00<00:00, 117MB/s]


Ultralytics 8.3.91 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=license_plate_detection.yaml, epochs=16, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=2, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_box

100%|██████████| 755k/755k [00:00<00:00, 23.5MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 111MB/s]


...

                   all        180        180          1          1      0.995      0.789



16 epochs completed in 0.070 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.91 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:02<00:00,  2.02it/s]


                   all        180        180          1          1      0.995      0.788
Speed: 0.4ms preprocess, 3.3ms inference, 0.0ms loss, 3.2ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x782c5b943a90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

Above values indicate that model has been trained well

In [ ]:
from google.colab import files

# Download the best-trained model
files.download("runs/detect/train/weights/best.pt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>